In [108]:
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
from tqdm import tqdm as tq
from collections import Counter
import math
import numpy as np
from heapq import *
import geocoder
import heapq

In [2]:
### Reading the dataset
data=pd.read_csv('final_dataset_2_rm_nan_url.csv')
data=data.drop('Unnamed: 0',axis=1)
data

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
0,City Hall Station,"Manhattan, New York",1824.0,8569.0,The first New York City subway was built and o...,"City Hall Station,Manhattan, New York, A beaut...","['Tunnel Number 3', 'African Burial Ground Nat...","31 Centre StNew York, New York, 10007United St...",40.7134,-74.0046,"['charding407', 'fosterc827', 'Mike H', 'erjef...","May 8, 2010",['30 Unexpected Places to Have a Joyful Advent...,"['Crystal Palace Subway', 'Moscow Metro Statio...",https://www.atlasobscura.com/places/city-hall-...
1,Highgate Cemetery,"London, England",2615.0,8179.0,"Opened in 1839, Highgate is one of London’s mo...","Highgate Cemetery,London, England, London's cr...","['Dick Whittington’s Cat', ""World's Largest Po...","Swain's Lane, HighgateLondon, England, N6Unite...",51.5675,-0.1483,"['thesecretcity', 'thedyerghoulhouse', 'juliap...","August 9, 2014","[""The World's Top 100 Wonders in 2018"", ""Londo...","['Jewett City Vampires', 'Tomb of the Mather F...",https://www.atlasobscura.com/places/highgate-c...
2,Leadenhall Market,"London, England",3133.0,7560.0,The ornate 19th-century painted roof and cobbl...,"Leadenhall Market,London, England, This ornate...","['The Cornhill Devils', ""London's Original and...","London, England, EC3VUnited Kingdom",51.5126,-0.0834,"['Julius Spada', 'Katecrimson', 'thesecretcity...","August 1, 2016","['The Ultimate Guide to Stunning, Surprising, ...","['Rivendell', 'Bagdad Cafe', 'Gare de la Ciota...",https://www.atlasobscura.com/places/leadenhall...
3,The Wave Organ,"San Francisco, California",2426.0,7400.0,"Located on a jetty in the San Francisco Bay, t...","The Wave Organ,San Francisco, California, A hu...","['The Stern of the Brigantine Galilee', 'Long ...","83 Marina Green DrSan Francisco, California, 9...",37.8085,-122.4401,"['routt1111', 'wefiddleboy14', 'Saal333', 'Nei...","November 21, 2008","[""Leonardo Nam's 16 Quirky Roadside Attraction...","['Sea Organ', 'Silent Green Kulturquartier', ""...",https://www.atlasobscura.com/places/wave-organ
4,Catacombes de Paris,"Paris, France",4442.0,7036.0,"In 2004, Parisian police were assigned to do a...","Catacombes de Paris,Paris, France, The vast, l...","['Sculptures de Décure', ""Jeannot's Floor"", 'A...","1 Place Denfert-RochereauParis, 75014France",48.8343,2.3322,"['marypippen', 'ramonrodz2212', 'mathewbrowne'...","February 13, 2009","['19 Catacombs Sure to Tingle Your Spine', ""Th...","['Ossario di San Martino', 'Leuk Charnel House...",https://www.atlasobscura.com/places/catacombes...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,Xul Solar Museum,"Buenos Aires, Argentina",129.0,405.0,With dreams of reforming and perfecting the un...,"Xul Solar Museum,Buenos Aires, Argentina, Xul ...","['Ricardo Rojas House Museum', 'Villa Freud', ...",Laprida 1212Buenos AiresArgentina,-34.5948,-58.4078,[],"September 12, 2016","[""A Linguistics Lover's Tour of the World""]","['Blenko Man', 'Ilana Goor Museum', 'Museo de ...",https://www.atlasobscura.com/places/xul-solar-...
7196,Klatovy Catacomb Mummies,"Klatovy, Czechia",49.0,405.0,In an underground crypt beneath a Jesuit churc...,"Klatovy Catacomb Mummies,Klatovy, Czechia, The...","['Drachenstich-Festspiele', 'Poledník Lookout ...",DenisovaKlatovyCzechia,49.3951,13.2918,[],"February 25, 2016","['19 Catacombs Sure to Tingle Your Spine', ""Bo...","['Rothwell Bone Crypt', 'Ossario di San Martin...",https://www.atlasobscura.com/places/klatovy-ca...
7197,Termesphere Gallery,"Spearfish, South Dakota",71.0,405.0,"Most artists paint on a a flat plane, but in t...","Termesphere Gallery,Spearfish, South Dakota, T...","['Fish Car No. 3', 'Mount Roosevelt Friendship...","1920 Christensen DriveSpearfish, South Dakota,...",44.4574,-103.8258,[],"August 19, 2014",NaN,NaN,https://www.atlasobscura.com/places/termespher...
7198,Wheeler 

In [3]:
def preprocess(text:str):
    ### Converting to lowercase
    text = text.lower()
    
    ### Removing Punctuation
    text_p = "".join([char for char in text if char not in string.punctuation])
    
    ### Converting strings to tokens of individual words in a complete string
    words = word_tokenize(text_p)
    
    ### Remove Stopwords
    stop_words = stopwords.words('english')
    filtered_words = [word for word in words if word not in stop_words]
    
    ### Get the root of the word i.e stemming
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in filtered_words]
    
    return stemmed

In [4]:
### Reading Stored inverted index
inverted_index_dataframe=pd.read_csv('inverted_index_tf_idf.csv')

inverted_index_dataframe=inverted_index_dataframe.drop('Unnamed: 0',axis=1)
for column in inverted_index_dataframe:
    
    if column=='Document_id' or column=='Doc_id_tf_idf':
        
        inverted_index_dataframe[column]=inverted_index_dataframe[column].apply(lambda row: eval(row) ) #converting strings to object type
inverted_index_dataframe

,Term_id,Document_id,Doc_id_tf_idf
0,-7165125546970577434,"[8650740614433979565, 6762175636761094086, -69...","[{8650740614433979565: 0.005076142131979695}, ..."
1,-77033085036089818,"[8650740614433979565, 7692525456237221879, -20...","[{8650740614433979565: 0.015228426395939087}, ..."
2,4204630629893213509,"[8650740614433979565, -8595611825221734725, -8...","[{8650740614433979565: 0.015228426395939087}, ..."
3,1325781193330695663,"[8650740614433979565, 6762175636761094086, 289...","[{8650740614433979565: 0.030456852791878174}, ..."
4,3196231265570269517,"[8650740614433979565, -8946468704458547891, -7...","[{8650740614433979565: 0.005076142131979695}, ..."
...,...,...,...
51787,-876772972226081506,[-708361252141436895],[{-708361252141436895: 0.008403361344537815}]
51788,-1721524412824808313,[-708361252141436895],[{-708361252141436895: 0.008403361344537815}]
51789,5801661624547006870,[1851989602745493214],[{1851989602745493214: 0.02097902097902098}]
51790,-109308169976446818,[1851989602745493214],[{1851989602745493214: 0.006993006993006993}]


In [5]:
### Reading Document data_frame containing key of the documents and modified version of original dataset
### in sense get those columns required for variable calculation
### Note:You might see some columns not being used.Have to find some metric for those columns

data_des=pd.read_csv('description_hash_key.csv')
data_des=data_des.drop('Unnamed: 0',axis=1)

for column in data_des:
    
    if column=='processed':
        
        data_des[column]=data_des[column].apply(lambda row: eval(row) ) #converting strings to object type
        
data_des['placeName']=data['placeName']
data_des['placeURL']=data['placeURL']
data_des['placeNearby']=data['placeNearby']
data_des['numPeopleVisited']=data['numPeopleVisited']
data_des['numPeopleWant']=data['numPeopleWant']
data_des['placeAlt']=data['placeAlt']
data_des['placeLong']=data['placeLong']
data_des


,placeDesc,processed,hash_key,placeName,placeURL,placeNearby,numPeopleVisited,numPeopleWant,placeAlt,placeLong
0,The first New York City subway was built and o...,"[first, new, york, citi, subway, built, oper, ...",8650740614433979565,City Hall Station,https://www.atlasobscura.com/places/city-hall-...,"['Tunnel Number 3', 'African Burial Ground Nat...",1824.0,8569.0,40.7134,-74.0046
1,"Opened in 1839, Highgate is one of London’s mo...","[open, 1839, highgat, one, london, ’, infam, c...",6762175636761094086,Highgate Cemetery,https://www.atlasobscura.com/places/highgate-c...,"['Dick Whittington’s Cat', ""World's Largest Po...",2615.0,8179.0,51.5675,-0.1483
2,The ornate 19th-century painted roof and cobbl...,"[ornat, 19thcenturi, paint, roof, cobbl, floor...",-5302886591338703343,Leadenhall Market,https://www.atlasobscura.com/places/leadenhall...,"['The Cornhill Devils', ""London's Original and...",3133.0,7560.0,51.5126,-0.0834
3,"Located on a jetty in the San Francisco Bay, t...","[locat, jetti, san, francisco, bay, wave, orga...",2892850636241140390,The Wave Organ,https://www.atlasobscura.com/places/wave-organ,"['The Stern of the Brigantine Galilee', 'Long ...",2426.0,7400.0,37.8085,-122.4401
4,"In 2004, Parisian police were assigned to do a...","[2004, parisian, polic, assign, train, exercis...",-6945605901378363543,Catacombes de Paris,https://www.atlasobscura.com/places/catacombes...,"['Sculptures de Décure', ""Jeannot's Floor"", 'A...",4442.0,7036.0,48.8343,2.3322
...,...,...,...,...,...,...,...,...,...,...
7195,With dreams of reforming and perfecting the un...,"[dream, reform, perfect, univers, argentin, ar...",1224246858281487530,Xul Solar Museum,https://www.atlasobscura.com/places/xul-solar-...,"['Ricardo Rojas House Museum', 'Villa Freud', ...",129.0,405.0,-34.5948,-58.4078
7196,In an underground crypt beneath a Jesuit churc...,"[underground, crypt, beneath, jesuit, church, ...",-4768850378645810701,Klatovy Catacomb Mummies,https://www.atlasobscura.com/places/klatovy-ca...,"['Drachenstich-Festspiele', 'Poledník Lookout ...",49.0,405.0,49.3951,13.2918
7197,"Most artists paint on a a flat plane, but in t...","[artist, paint, flat, plane, termespher, galle...",2318087413727924295,Termesphere Gallery,https://www.atlasobscura.com/places/termespher...,"['Fish Car No. 3', 'Mount Roosevelt Friendship...",71.0,405.0,44.4574,-103.8258
7198,At least six mountains in the southwestern Uni...,"[least, six, mountain, southwestern, unit, sta...",-708361252141436895,Wheeler Peak,https://www.atlasobscura.com/places/wheeler-peak,"['El Salto del Agua Waterfall', 'Taos Pueblo',...",241.0,405.0,36.6054,-105.4302


In [6]:
### Vocabulary prepared from the dataset based on the long description of the place

vocabulary_df=pd.read_csv('vocabulary.csv')
vocabulary_df.rename(columns={'Unnamed: 0':'Term_id'},inplace=True)
# vocabulary_df.index=vocabulary_df['Term_id']
vocabulary_df=vocabulary_df.dropna()
vocabulary_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 51790 entries, 0 to 51791
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Term_id       51790 non-null  object
 1   term_hashkey  51790 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.2+ MB


In [98]:
def get_document_id_tf_idf_vector(query,vocabulary_df,inverted_index_dataframe):
    
    #### Main Objective:Get the Document_id's of the matched documents and it's tf_id representation vectors for all the documents
    
    query_processed=preprocess(query)

    ### Get the term_id from vocabulary
    term_id=[vocabulary_df[vocabulary_df.Term_id==i].term_hashkey.values[0] for i in query_processed]

    ## Get the dictionary of {document_id:tf_idf scores} for each terms.
    query_documents=[inverted_index_dataframe[inverted_index_dataframe.Term_id==i].Doc_id_tf_idf.values[0] for i in term_id]

    #### get the document_id for each of the term
    query_documents_keys=[]
    for i in range(len(query_documents)):
        list_query=[list(i.keys())[0] for i in query_documents[i]]
        query_documents_keys.append(list_query)

    ####Find the intersection of document_id's for all the terms to get those document_id which contains all the queries
    if len(query_documents_keys)>1:
        for i in range(1,len(query_documents_keys)):
            if i==1:
                document_id=set(query_documents_keys[i-1]).intersection(query_documents_keys[i])

            else:
                document_id=document_id.intersection(query_documents_keys[i])

    else:
        document_id=query_documents_keys[0]

    ### Based on the retrieved common document_id get the values for tf_idf score for each terms.
    query_documents_values=[]
    for i in range(len(query_documents)):
        list_query=[list(i.values())[0] for i in query_documents[i] if list(i.keys())[0] in document_id]
        query_documents_values.append(list_query)

    #### Create document vector whose individual length corresponds the number of queries.
    query_document=zip(*query_documents_values)
    query_document_vector=[i for i in query_document]
    
    return list(document_id),query_document_vector



In [42]:

def get_query_vector(query,vocabulary_df,inverted_index_dataframe):
    
    ## Pre-processing query
    query_processed=preprocess(query)
    
    ### Get the term_id from vocabulary
    term_id=[vocabulary_df[vocabulary_df.Term_id==i].term_hashkey.values[0] for i in query_processed]


    ## Get the documents index from inverted index
    query_idf=[np.log(len(data_des)/len(inverted_index_dataframe[inverted_index_dataframe.Term_id==i].Doc_id_tf_idf.values[0])) for i in term_id]
    query_tf=[1/len(query_processed) for i in query_processed]

    query_vector=np.multiply(query_idf,query_tf)
    return query_vector

In [127]:
def calculate_cosine_similarity(query_vector:list,query_document_vector):
 
    z=np.dot(query_vector,query_document_vector)/(np.linalg.norm(query_vector)*np.linalg.norm(query_document_vector))

#         z=z.sum()/math.sqrt(len(query_processed)) ### Take care of the parameterisation of the calculation,it would lenght of the quer

    z=z.sum()        
    return z

In [58]:
# !pip install geocoder
def get_current_position():
    current_position=geocoder.ip('me')
    return current_position

In [56]:
def angleFromCoordinate(lat1, long1, lat2, long2): ###Reference:https://stackoverflow.com/questions/3932502/calculate-angle-between-two-latitude-longitude-points
    #### Get angle between two co-ordinates
    dLon = (long2 - long1)

    y = math.sin(dLon) * math.cos(lat2)
    x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(dLon)

    brng = math.atan2(y, x)

    brng = math.degrees(brng)
    brng = (brng + 360) % 360
    brng = 360 - brng # count degrees clockwise - remove to make counter-clockwise

    return brng

In [57]:
def getangle(c1,c2):
    return angleFromCoordinate(c1.latlng[0], c1.latlng[1], c2[0], c2[1])

In [63]:
def get_postion_score(g,coord):
    #### Position score of the place
#     g=get_current_position()
    
    angle_score=(360-getangle(g,coord))/360
   
    return  angle_score

In [123]:

def get_visit_score(visit,max_visit):
### Get score for either People visited or people want to visit
    return visit/max_visit

In [158]:
def ask_user_information():
## Asking additional information from the user
    print('1. Do you want  places to be closer to you?')
    print('2. Do you want to see based on number of people visited the place?')
    print('3. Do you want to see based on number people want to go the place?')
    x=int(input('Select from the given option:'))
    return x

In [164]:
def Search_engine_top_k(query,vocabulary_df,inverted_index_dataframe,data_des,k):
    i=ask_user_information()
    if i==1:
        ### Get document id common to all queries and it's tf_id representation
        document_id,query_document_vector=get_document_id_tf_idf_vector(query,vocabulary_df,inverted_index_dataframe)
        
        ### Get Query Vector
        query_vector=get_query_vector(query,vocabulary_df,inverted_index_dataframe)
        
        ### Scoring Metric defined for this choice
        metric=[]
        
        #### Converting list to heap
        heapq.heapify(metric)
        
        ### Get my current position
        g=get_current_position()
        
        ### Variable for top_k documents
        max_length=k
        
        #### Iterate over the document id's and get similarity score & Position Score for each document to get the final score
        for i,j in enumerate(document_id):
                     ### Get co-ordinates of the place   
                     coord=data_des[data_des.hash_key.isin([j])].loc[:,['placeAlt','placeLong']].values[0]
                    
                     ### Find the angle with respect to the location
                     position_score=get_postion_score(g,coord)
                    
                     ### Cosine Similarity based on tf_idf representation of the document
                     similarity=calculate_cosine_similarity(query_vector,query_document_vector[i])
                     ### Final Score for the document
                     score=position_score*similarity
                     ### Storing upto the size of the heap
                     if i<max_length:
                        heappush(metric,(score,j))

                    ## Replacing the minumum element from the heap(Gaol is to maintain the top scores)
                     else:
                        if score>metric[0][0]:
                            heapreplace(metric,(score,j))
                            
        ### Get the sorted order of the metric: Each element is (Score,Document_id)        
        metric=heapq.nlargest(k,metric)
        
        ### Get the Final DataFrame
        for i,j in enumerate(metric):
            if i==0:
                final_df=data_des[data_des.hash_key.isin([j[1]])]
                final_df=final_df.assign(Score=j[0])
            else:
                x=data_des[data_des.hash_key.isin([j[1]])]
                x=x.assign(Score=j[0])
                final_df=pd.concat([final_df,x])
                
        return final_df.drop(['processed','hash_key','placeNearby','numPeopleVisited','numPeopleWant','placeAlt','placeLong'],axis=1)
    
    #### The syntax of the code is quite similar as present for i==1.Refer to that to get more information.
    elif i==2:
        ###Score based on Number of People who visited the place
        document_id,query_document_vector=get_document_id_tf_idf_vector(query,vocabulary_df,inverted_index_dataframe)
        query_vector=get_query_vector(query,vocabulary_df,inverted_index_dataframe)
        metric=[]
        heapq.heapify(metric)
        max_visit=data_des[data_des.hash_key.isin(document_id)].numPeopleVisited.max()

        max_length=k
        for i,j in enumerate(document_id):

                     visit=data_des[data_des.hash_key.isin([j])].loc[:,['numPeopleVisited']].values[0]
                     visit_score=get_visit_score(visit,max_visit)

                     similarity=calculate_cosine_similarity(query_vector,query_document_vector[i])

                     score=visit_score*similarity

                     if i<max_length:
                        heappush(metric,(score,j))


                     else:
                        if score>metric[0][0]:
                            heapreplace(metric,(score,j))
        metric=heapq.nlargest(k,metric)
        for i,j in enumerate(metric):
            if i==0:
                final_df=data_des[data_des.hash_key.isin([j[1]])]
                final_df=final_df.assign(Score=j[0])
            else:
                x=data_des[data_des.hash_key.isin([j[1]])]
                x=x.assign(Score=j[0])
                final_df=pd.concat([final_df,x])
        
        return final_df.drop(['processed','hash_key','placeNearby','numPeopleVisited','numPeopleWant','placeAlt','placeLong'],axis=1)

    elif i==3:
        ###Score based on Number of People who want to visit
        document_id,query_document_vector=get_document_id_tf_idf_vector(query,vocabulary_df,inverted_index_dataframe)
        query_vector=get_query_vector(query,vocabulary_df,inverted_index_dataframe)
        metric=[]
        heapq.heapify(metric)
        max_visit_want=data_des[data_des.hash_key.isin(document_id)].numPeopleWant.max()

        max_length=k
        for i,j in enumerate(document_id):
           
             visit_want=data_des[data_des.hash_key.isin([j])].loc[:,['numPeopleWant']].values[0]
             visit_want_score=get_visit_score(visit_want,max_visit_want)
            
             similarity=calculate_cosine_similarity(query_vector,query_document_vector[i])
             
             score=visit_want_score*similarity
             
             if i<max_length:
                heappush(metric,(score,j))
                
                
             else:
                if score>metric[0][0]:
                    heapreplace(metric,(score,j))
        metric=heapq.nlargest(k,metric)
        for i,j in enumerate(metric):
                if i==0:
                    final_df=data_des[data_des.hash_key.isin([j[1]])]
                    final_df=final_df.assign(Score=j[0])
                else:
                    x=data_des[data_des.hash_key.isin([j[1]])]
                    x=x.assign(Score=j[0])
                    final_df=pd.concat([final_df,x])
        return final_df.drop(['processed','hash_key','placeNearby','numPeopleVisited','numPeopleWant','placeAlt','placeLong'],axis=1)

        
    
        

In [163]:
Search_engine_top_k(query,vocabulary_df,inverted_index_dataframe,data_des,50)

1. Do you want  places to be closer to you?
2. Do you want to see based on number of people visited the place?
3. Do you want to see based on number people want to go the place?
Select from the given option:3


,placeDesc,placeName,placeURL,Score
141,The dime or dime store museum is by all accoun...,Museum of the Weird,https://www.atlasobscura.com/places/museum-weird,0.738262
59,The Salem witchcraft trials took place between...,The Witch House of Salem,https://www.atlasobscura.com/places/witch-hous...,0.737545
173,Las Vegas is an oasis in the desert where no o...,National Atomic Testing Museum,https://www.atlasobscura.com/places/national-a...,0.628333
224,Ripe for quiet reflection and simply awe-inspi...,Canyons of the Ancients,https://www.atlasobscura.com/places/canyons-of...,0.571028
329,Like an elegant antiques mall gone horribly wr...,Uncommon Objects,https://www.atlasobscura.com/places/uncommon-o...,0.559191
320,Often cited as having a place among the great ...,The Natural Bridge,https://www.atlasobscura.com/places/the-natura...,0.544348
356,The death of a pet can inspire a number of rea...,KattenKabinet,https://www.atlasobscura.com/places/kattenkabinet,0.543457
414,Built sometime between 1550 and 1292 BC as a c...,Petra,https://www.atlasobscura.com/places/petra,0.432925
690,"“What in Sam Hill?” This odd expression, of wh...",Maryhill Museum and Stonehenge,https://www.atlasobscura.com/places/maryhill-m...,0.408592
360,"In the 1995 hit film Outbreak , residents of t...",CDC Museum,https://www.atlasobscura.com/places/cdc-museum,0.403348
